In [19]:
from __future__ import annotations
import pandas as pd
from pathlib import Path
from typing import Dict, List
import io
import os
import numpy as np

In [20]:
# settings
log_dir = '/var/avl_logs/current/log/'

# logs to be read
# log_names = ['data_collector.log']
# leave empty to read all files
log_names = []

log_time_window = 0.1

data_collector_units = ['NAN', 'rad', 'rad', 'rad', 'm/s', 'm/s', 'm/s', 'rad', 'rad', 'm', 'rad', 'rad', 'm', 'm', 'm', 'm', 'rad/s', 'rad/s', 'rad/s', 'm/s^2', 'm/s^2', 'm/s^2', 'NAN']

# output csv filename
log_dst_csv_name = 'avl_log.csv'

In [21]:
def read_log(log_file: Path) -> Dict[str, pd.DataFrame]:
    log_strs = {}
    log_dfs = {}
    with open(log_file, "r") as f:
        data_lines = [line for line in f.readlines() if "[DAT]" in line]
        
        # TODO currently use this to prevent reading useless files
        if len(data_lines) < 20:
            return {}

        for line in data_lines:
            stream = os.path.basename(log_file)[0: -4] + '_' + line.strip().split(' ')[3]
            if stream not in log_strs:
                log_strs[stream] = []

            data = line.strip().split(' ')
            time = data[0].replace("[", "").replace("]", "")
            log_strs[stream].append([time] + data[4:])

        for stream, data in log_strs.items():
            names = data.pop(0)[1:]
            units = data.pop(0)[1:]

            # VectorXd y;          // Measurement vector
            # VectorXd y_bar;      // Estimated measurement vector
            # MatrixXd P_yy;       // Measurement covariance matrix
            # VectorXd innovation; // Measurement innovation (error) vector
            # VectorXd threshold;  // Measurement rejection threshold multiplier
            # bool accepted;       // Measurement accepted or rejected
            # # Special handling for inertial_nav_node.log
            # if "sats" in names
            if "sats" in names:
                sats_index = names.index("sats")
                names = names[:sats_index] + ["num_sats", "y_1", "y_2", "y_3", "y_bar_1", "y_bar_2", "y_bar_3", "P_yy_1",  "P_yy_2",  "P_yy_3",  "P_yy_4",  "P_yy_5",  "P_yy_6",  "P_yy_7",  "P_yy_8",  "P_yy_9", "innovation_1", "innovation_2", "innovation_3", "threshold_1", "threshold_2", "threshold_3", "accepted"] + names[sats_index + 1:]
            if "sats" in units:
                sats_index = units.index("sats")
                units = units[:sats_index] + ["?"]*23 + units[sats_index + 1:]

            col_names = ["time"] + [f"[{stream}]{name}_{unit}" for name, unit in zip(names, units)]
            df = pd.DataFrame(data, columns=col_names)
            if len(df) > 0:
                log_dfs[stream] = df

    return log_dfs

def read_data_collector_log(log_file: Path) -> Dict[str, pd.DataFrame]:
    stream = 'data_collector'
    log_df = pd.read_csv(log_file, delim_whitespace=True)
    log_df = log_df.drop(index=0, axis=0).reset_index(drop=True)
    columns = list(log_df.columns)
    if len(columns) - 1 != len(data_collector_units):
        print('data_collector_units not match error')
        return
    
    columns[0] = 'time'
    for i in range(0, len(data_collector_units)):
            columns[i + 1] = f'[{stream}]{columns[i + 1]}_{data_collector_units[i]}'
    log_df.columns = columns
    
    log_dfs = {}
    log_dfs[stream] = log_df
    return log_dfs


# Read required the log files and put their contents into the log_lines dict
# Grab required files in log folder that end in .log
if len(log_names) == 0:
    log_files = [log_file for log_file in Path(log_dir).iterdir() if log_file.is_file() and log_file.suffix == '.log']
else:
    log_files = [Path(log_dir + log_name) for log_name in log_names if Path(log_dir + log_name).is_file and Path(log_dir + log_name).suffix == '.log']

log_raw_data = {}
for log_file in log_files:
    if os.path.basename(log_file) == 'data_collector.log':
        log_dfs = read_data_collector_log(log_file)
    elif os.path.basename(log_file) == 'inertial_nav_node.log': # TODO currently don't know why this can't read this file
        # log_dfs = {}
        log_dfs = read_log(log_file)
    elif os.path.basename(log_file) == 'ethernet_channel_node.log': # TODO this file's format is different and I think this file is useless
        log_dfs = {}
    else:
        log_dfs = read_log(log_file)
    for stream, data in log_dfs.items():
        log_raw_data[stream] = log_dfs[stream]
        display(log_raw_data[stream])


,time,[dvl_sim_node_[v_eb_b]]v_x_m/s,[dvl_sim_node_[v_eb_b]]v_y_m/s,[dvl_sim_node_[v_eb_b]]v_z_/ms
0,1679338479.360456944,0.020062765,-0.007288369,0.011366872
1,1679338479.384000540,-0.007110125,-0.006765954,-0.007378259
2,1679338479.405364752,0.016735300,0.013830196,-0.012832922
3,1679338479.431869984,0.023161970,0.008400622,-0.003496241
4,1679338479.455102205,0.009280731,-0.003397324,-0.006947636
...,...,...,...,...
533,1679338491.971769333,2.002794495,0.012382881,0.001925657
534,1679338491.993350744,2.004093207,-0.004817983,-0.004176788
535,1679338492.015007496,2.008413981,0.005931595,0.010389620
536,1679338492.035955906,1.999709380,0.026526254,0.017472762


,time,[gps_sim_node_[gps]]lat_rad,[gps_sim_node_[gps]]lon_rad,[gps_sim_node_[gps]]alt_m,[gps_sim_node_[gps]]ground\_speed_m/s,[gps_sim_node_[gps]]track\_angle_rad,[gps_sim_node_[gps]]fix\_quality_NA,[gps_sim_node_[gps]]num\_sats_num,[gps_sim_node_[gps]]hdop_NA
0,1679338479.370952845,0.646784024,-1.406986608,-1.2,0.0000,0.0000,1,10,1
1,1679338479.393488169,0.646783924,-1.406986132,-0.2,0.0000,0.0000,1,10,1
2,1679338479.419703007,0.646783678,-1.406986084,-2.0,0.0000,0.0000,1,10,1
3,1679338479.443169594,0.646783680,-1.406986463,-0.2,0.0000,0.0000,1,10,1
4,1679338479.466058969,0.646783675,-1.406985905,0.8,0.0000,0.0000,1,10,1
...,...,...,...,...,...,...,...,...,...
532,1679338491.962120533,0.646791185,-1.406982184,13.8,1.9979,0.2356,1,10,1
533,1679338491.981745958,0.646792304,-1.406982788,14.0,1.9980,0.2351,1,10,1
534,1679338492.003340721,0.646791968,-1.406982710,16.1,1.9980,0.2346,1,10,1
535,1679338492.024300337,0.646792273,-1.406981762,13.2,1.9981,0.2341,1,10,1


,time,[pid_attitude_control_node_[roll_pid]]roll\_input_rad,[pid_attitude_control_node_[roll_pid]]roll\_setpoint_rad,[pid_attitude_control_node_[roll_pid]]fin\_angle\_output_rad
0,1679338481.475810289,0.0002,nan,0.0000
1,1679338481.488079071,0.0002,0.0000,-0.0002
2,1679338481.501100779,0.0002,0.0000,-0.0002
3,1679338481.513388872,0.0006,0.0000,-0.0006
4,1679338481.525545359,0.0006,0.0000,-0.0006
...,...,...,...,...
901,1679338492.165327787,-0.0063,0.0000,0.0063
902,1679338492.178188324,-0.0063,0.0000,0.0063
903,1679338492.187994957,-0.0063,0.0000,0.0063
904,1679338492.201057196,-0.0063,0.0000,0.0063


,time,[pid_attitude_control_node_[pitch_pid]]pitch\_input_rad,[pid_attitude_control_node_[pitch_pid]]pitch\_setpoint_rad,[pid_attitude_control_node_[pitch_pid]]elevator\_angle\_output_rad
0,1679338481.475865841,-0.0007,nan,0.3491
1,1679338481.488116980,-0.0007,0.2618,0.3491
2,1679338481.501156330,-0.0009,0.2618,0.3491
3,1679338481.513430357,-0.0006,0.2618,0.3491
4,1679338481.525581837,-0.0004,0.2618,0.3491
...,...,...,...,...
901,1679338492.165357590,0.2632,0.2618,0.0177
902,1679338492.178255558,0.2632,0.2618,0.0177
903,1679338492.188021898,0.2632,0.2618,0.0176
904,1679338492.201097727,0.2632,0.2618,0.0176


,time,[pid_attitude_control_node_[yaw_pid]]yaw\_input_rad,[pid_attitude_control_node_[yaw_pid]]yaw\_setpoint_rad,[pid_attitude_control_node_[yaw_pid]]rudder\_angle\_output_rad
0,1679338481.475888014,0.7854,0.0000,-0.3491
1,1679338481.488137484,0.7854,0.0000,-0.3491
2,1679338481.501177549,0.7854,0.0000,-0.3491
3,1679338481.513452053,0.7854,0.0000,-0.3491
4,1679338481.525607347,0.7855,0.0000,-0.3491
...,...,...,...,...
901,1679338492.165370941,0.0067,0.0000,-0.0090
902,1679338492.178271055,0.0067,0.0000,-0.0090
903,1679338492.188036442,0.0067,0.0000,-0.0090
904,1679338492.201121092,0.0067,0.0000,-0.0090


...

,time,[dynamics_manager_node_[eta]]x_m,[dynamics_manager_node_[eta]]y_m,[dynamics_manager_node_[eta]]z_m,[dynamics_manager_node_[eta]]\phi_rad,[dynamics_manager_node_[eta]]\theta_rad,[dynamics_manager_node_[eta]]\psi_rad
0,1679338479.349060297,0.000000,0.000000,0.000000,0.000000,0.000000,0.785398
1,1679338479.350879669,0.000000,0.000000,0.000000,0.000000,0.000000,0.785398
2,1679338479.353676081,0.000000,0.000000,0.000000,0.000000,0.000000,0.785398
3,1679338479.354995966,0.000000,0.000000,0.000000,0.000000,0.000000,0.785398
4,1679338479.356763840,0.000000,0.000000,0.000000,0.000000,0.000000,0.785398
...,...,...,...,...,...,...,...
5069,1679338492.258903265,55.560917,20.296898,-14.911872,-0.001379,0.274241,0.233679
5070,1679338492.260571480,55.579137,20.301278,-14.917049,-0.001377,0.274249,0.233633
5071,1679338492.262245655,55.597339,20.305652,-14.922220,-0.001375,0.274258,0.233587
5072,1679338492.264510155,55.615523,20.310021,-14.927385,-0.001372,0.274266,0.233541


,time,[dynamics_manager_node_[eta_dot]]x\_dot_m/s,[dynamics_manager_node_[eta_dot]]y\_dot_m/s,[dynamics_manager_node_[eta_dot]]z\_dot_m/s,[dynamics_manager_node_[eta_dot]]\phi\_dot_rad/s,[dynamics_manager_node_[eta_dot]]\theta\_dot_rad/s,[dynamics_manager_node_[eta_dot]]\psi\_dot_rad/s
0,1679338479.349084139,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,1679338479.350900650,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,1679338479.353698969,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,1679338479.355030298,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,1679338479.356783390,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...
5069,1679338492.258922100,1.823944,0.438501,-0.518171,0.000224,0.000805,-0.004609
5070,1679338492.260585308,1.822068,0.437960,-0.517628,0.000223,0.000828,-0.004598
5071,1679338492.262274504,1.820196,0.437419,-0.517087,0.000222,0.000850,-0.004587
5072,1679338492.264523745,1.818328,0.436880,-0.516547,0.000221,0.000871,-0.004577


,time,[dynamics_manager_node_[p_b]]lat0_rad,[dynamics_manager_node_[p_b]]lon0_rad,[dynamics_manager_node_[p_b]]alt0_m,[dynamics_manager_node_[p_b]]lat_rad,[dynamics_manager_node_[p_b]]lon_rad,[dynamics_manager_node_[p_b]]alt_m
0,1679338479.349161625,0.646783695,-1.406986268,0.00,0.646783695,-1.406986268,0.00
1,1679338479.350926876,0.646783695,-1.406986268,0.00,0.646783695,-1.406986268,0.00
2,1679338479.353731155,0.646783695,-1.406986268,0.00,0.646783695,-1.406986268,0.00
3,1679338479.355088949,0.646783695,-1.406986268,0.00,0.646783695,-1.406986268,0.00
4,1679338479.356811047,0.646783695,-1.406986268,0.00,0.646783695,-1.406986268,0.00
...,...,...,...,...,...,...,...
5069,1679338492.258947372,0.646783695,-1.406986268,0.00,0.646792434,-1.406982286,14.91
5070,1679338492.260603666,0.646783695,-1.406986268,0.00,0.646792436,-1.406982286,14.92
5071,1679338492.262307644,0.646783695,-1.406986268,0.00,0.646792439,-1.406982285,14.93
5072,1679338492.264541864,0.646783695,-1.406986268,0.00,0.646792442,-1.406982284,14.93


,time,"[dynamics_manager_node_[w_ib_b]]\omega_{ib,x}^b_rad/s","[dynamics_manager_node_[w_ib_b]]\omega_{ib,y}^b_rad/s","[dynamics_manager_node_[w_ib_b]]\omega_{ib,z}^b_rad/s"
0,1679338479.349191904,0.000041,-0.000041,-0.000044
1,1679338479.350950241,0.000041,-0.000041,-0.000044
2,1679338479.353752375,0.000041,-0.000041,-0.000044
3,1679338479.355131388,0.000041,-0.000041,-0.000044
4,1679338479.356835842,0.000041,-0.000041,-0.000044
...,...,...,...,...
5069,1679338492.258965015,0.001539,0.000797,-0.004462
5070,1679338492.260615826,0.001535,0.000820,-0.004452
5071,1679338492.262324333,0.001531,0.000842,-0.004442
5072,1679338492.264554501,0.001527,0.000863,-0.004432


,time,"[dynamics_manager_node_[f_ib_b]]f_{ib,x}^b_m/s^2","[dynamics_manager_node_[f_ib_b]]f_{ib,y}^b_m/s^2","[dynamics_manager_node_[f_ib_b]]f_{ib,z}^b_m/s^2"
0,1679338479.349215031,0.000000,0.000000,-9.799105
1,1679338479.350969553,0.000000,0.000000,-9.799105
2,1679338479.353771448,0.000000,0.000000,-9.799105
3,1679338479.355149746,0.000000,0.000000,-9.799105
4,1679338479.356892586,0.000000,0.000000,-9.799105
...,...,...,...,...
5069,1679338492.258981705,2.450890,0.003643,-9.433368
5070,1679338492.260627747,2.451391,0.003651,-9.433390
5071,1679338492.262350798,2.451893,0.003659,-9.433410
5072,1679338492.264566183,2.452396,0.003667,-9.433428


In [22]:
log_df = None
log_df_created = False
log_min_time = max([float(raw_df.at[0, 'time']) for _, raw_df in log_raw_data.items()])
log_max_time = min([float(raw_df.at[len(raw_df) - 1, 'time']) for _, raw_df in log_raw_data.items()])

for stream, raw_df in log_raw_data.items():
    log_time = log_min_time
    parsed_df = pd.DataFrame(columns=raw_df.columns)
    cur_row = 0
    
    while log_time < log_max_time:
        while float(raw_df.at[cur_row, 'time']) >= log_time and log_time < log_max_time:
            log_time += log_time_window
            parsed_df.loc[len(parsed_df)] = [str(log_time)] + list(raw_df.loc[cur_row][1:])
        else:
            cur_row += 1
    
    if log_df_created:
        log_df = pd.concat([log_df, parsed_df.drop('time', axis=1)], axis=1)
        print(f'finished processing {stream}')
    else:
        log_df = parsed_df
        log_df_created = True

display(log_df)
log_df.to_csv(log_dst_csv_name, index=False)

finished processing gps_sim_node_[gps]
finished processing pid_attitude_control_node_[roll_pid]
finished processing pid_attitude_control_node_[pitch_pid]
finished processing pid_attitude_control_node_[yaw_pid]
finished processing pid_attitude_control_node_[roll_info]
finished processing pid_attitude_control_node_[pitch_info]
finished processing pid_attitude_control_node_[yaw_info]
finished processing pid_attitude_control_node_[fins]
finished processing depth_sim_node_[depth]
finished processing data_collector
finished processing actuators_node_[actuators]
finished processing auv_dynamics_node_[nu]
finished processing auv_dynamics_node_[eta]
finished processing auv_dynamics_node_[tau]
finished processing auv_dynamics_node_[nu_dot]
finished processing rpm_sim_node_[rpm]
finished processing rpm_control_node_[pid]
finished processing rpm_control_node_[info]
finished processing inertial_nav_node_[imu]
finished processing inertial_nav_node_[gps]
finished processing inertial_nav_node_[mag]
fi

,time,[dvl_sim_node_[v_eb_b]]v_x_m/s,[dvl_sim_node_[v_eb_b]]v_y_m/s,[dvl_sim_node_[v_eb_b]]v_z_/ms,[gps_sim_node_[gps]]lat_rad,[gps_sim_node_[gps]]lon_rad,[gps_sim_node_[gps]]alt_m,[gps_sim_node_[gps]]ground\_speed_m/s,[gps_sim_node_[gps]]track\_angle_rad,[gps_sim_node_[gps]]fix\_quality_NA,...,[dynamics_manager_node_[p_b]]alt0_m,[dynamics_manager_node_[p_b]]lat_rad,[dynamics_manager_node_[p_b]]lon_rad,[dynamics_manager_node_[p_b]]alt_m,"[dynamics_manager_node_[w_ib_b]]\omega_{ib,x}^b_rad/s","[dynamics_manager_node_[w_ib_b]]\omega_{ib,y}^b_rad/s","[dynamics_manager_node_[w_ib_b]]\omega_{ib,z}^b_rad/s","[dynamics_manager_node_[f_ib_b]]f_{ib,x}^b_m/s^2","[dynamics_manager_node_[f_ib_b]]f_{ib,y}^b_m/s^2","[dynamics_manager_node_[f_ib_b]]f_{ib,z}^b_m/s^2"
0,1679338481.5774896,-0.007655670,0.037512149,0.005548887,0.646784061,-1.406986771,0.7,0.0000,0.0000,1,...,0.00,0.646783695,-1.406986268,0.00,0.000041,-0.000041,-0.000044,0.000000,0.000000,-9.799105
1,1679338481.6774895,-0.008301768,-0.023045217,0.007149922,0.646783447,-1.406986154,-0.1,0.0009,0.7854,1,...,0.00,0.646783695,-1.406986268,0.00,0.000041,-0.000049,-0.000044,0.004193,-0.000000,-9.799106
2,1679338481.7774894,-0.011318949,-0.014492343,0.000547967,0.646783903,-1.406986399,-1.0,0.0045,0.7854,1,...,0.00,0.646783695,-1.406986268,0.00,0.000041,-0.000092,-0.000044,0.013219,0.000000,-9.799106
3,1679338481.8774893,0.013882067,0.000875285,0.005572290,0.646783993,-1.406987014,0.1,0.0139,0.7853,1,...,0.00,0.646783696,-1.406986268,0.00,0.000042,-0.000215,-0.000047,0.026544,0.000003,-9.799100
4,1679338481.9774892,0.012339300,-0.006966742,0.021806358,0.646783243,-1.406985920,1.6,0.0308,0.7852,1,...,0.00,0.646783697,-1.406986266,0.00,0.000043,-0.000380,-0.000064,0.042818,0.000013,-9.799074
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97,1679338491.2774804,2.008560277,0.007270895,0.010876432,0.646791366,-1.406982881,11.8,1.9959,0.2541,1,...,0.00,0.646791187,-1.406982679,12.65,0.001668,-0.000642,-0.005176,2.658568,0.004218,-9.430273
98,1679338491.3774803,1.996284870,0.004255184,0.012688062,0.646791598,-1.406982432,12.0,1.9962,0.2523,1,...,0.00,0.646791307,-1.406982640,12.87,0.001535,-0.000111,-0.005051,2.657379,0.004520,-9.431609
99,1679338491.4774802,1.995551461,0.025063072,0.000508805,0.646791594,-1.406983279,12.7,1.9966,0.2496,1,...,0.00,0.646791448,-1.406982594,13.13,0.001501,0.000128,-0.005095,2.658482,0.004021,-9.432056
100,1679338491.57748,2.023046036,0.004535207,0.016991917,0.646790978,-1.406982431,12.3,1.9968,0.2475,1,...,0.00,0.646791577,-1.406982552,13.36,0.001476,0.000137,-0.005496,2.658851,0.004459,-9.431953
